<a href="https://colab.research.google.com/github/daffamaulana03/Fuzzy-Logic/blob/master/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy
import pandas

# **BACA DATA**

In [0]:
def readFile():
    data = open("influencers.csv", "r")
    datata = data.readlines()
    
    array = []


    for i in range(1, len(datata)):
      array.append(datata[i].replace('\n','').split(','))
    # print(array[1][1])
    # print(len(array))
    return array

# **FUZZIFIKASI**

Atribut Followers

In [60]:
def fuzzificationFollowers():
  data = readFile()
  followers = []
  # FOLLOWERS

  # Upper	: >80K NOT Upper : <=55K
  # Middle	: 40K<=MID<=50K NOT Mid : <=20K & >80K
  # Bottom	: <=20K NOT Bottom : >45K

  Upper = 80000
  NotUpper = 55000
  Middle1 = 40000
  Middle2 = 50000
  Bottom = 20000
  NotBottom = 45000

  for i in range(5):
    # for j in range(len(data[i])):
    x = int(data[i][1])
    #UPPER
    if (x > Upper):
      Up = 1
    elif (x <= NotUpper):
      Up = 0
    else:
      Up = round((x-NotUpper)/(Upper-NotUpper),2)

    #BOTTOM
    if (x <= Bottom):
      Bot = 1
    elif (x > NotBottom):
      Bot = 0
    else:
      Bot = round((NotBottom-x)/(NotBottom-Bottom),2)

    #MIDDLE
    if ((x > Middle1) & (x <= Middle2)):
      Mid = 1
    elif ((x <= Bottom) or (x > Upper)):
      Mid = 0
    else:
      if (x > Middle2):
        Mid = round((Upper-x)/(Upper-Middle2),2)
      elif (x <= Middle1):
        Mid = round((x-Bottom)/(Middle1-Bottom),2)
    # print(i+1,Up)
    # print(i+1,Mid)
    # print(i+1,Bot)  
    foll = []
    foll.append(Up)
    foll.append(Mid)
    foll.append(Bot)

    followers.append(foll)
  return followers
fuzzificationFollowers()

[[0, 0, 1], [0, 0.91, 0.27], [0, 0, 1], [0, 1, 0.05], [0, 0, 1]]

Atribut Engagement Rate

In [59]:
def fuzzificationEngageRate():
  data = readFile()
  EngageRate = []
  # ENGAGEMENT RATE

  # High	: >8 NOT High : <=5.5
  # Average	: 4<AVG<5 NOT Mid : <=2 & >8
  # Low	: <=2 NOT Low : >4.5
  High = 8
  NotHigh = 5.5
  Average1 = 4
  Average2 = 5
  Low = 2
  NotLow = 4.5

  for i in range(5):
    # for j in range(len(data[i])):
    x = float(data[i][2])
    #HIGH
    if (x > High):
      Hi = 1
    elif (x <= NotHigh):
      Hi = 0
    else:
      Hi = round((x-NotHigh)/(High-NotHigh),2)

    #LOW
    if (x <= Low):
      Lo = 1
    elif (x > NotLow):
      Lo = 0
    else:
      Lo = round((NotLow-x)/(NotLow-Low),2)

    #AVERAGE
    if ((x > Average1) & (x <= Average2)):
      Avg = 1
    elif ((x <= Low) or (x > High)):
      Avg = 0
    else:
      if (x > Average2):
        Avg = round((High-x)/(High-Average2),2)
      elif (x <= Average1):
        Avg = round((x-Low)/(Average1-Low),2)
    # print(i+1,Up)
    # print(i+1,Mid)
    # print(i+1,Bot)  
    rate = []
    rate.append(Hi)
    rate.append(Avg)
    rate.append(Lo)

    EngageRate.append(rate)
  return EngageRate
fuzzificationEngageRate()

[[0, 0, 1], [0.12, 0.73, 0], [0, 0, 1], [0, 0, 1], [0, 0.75, 0.4]]

# **INFERENSI**

In [61]:
def inferensi():
  # FOLLOWERS	| ENGAGEMENT RATE	|	SCORE
  # Upper		  | High			      | medium
  # Upper		  |  Average			  | medium
  # Upper		  |  Low			      | micro
    
  # Middle		|  High			      | medium
  # Middle		|  Average			  | micro
  # Middle		|  Low			      | micro

  # Bottom		|  High			      | micro
  # Bottom		|  Average			  | micro
  # Bottom		|  Low			      | nano

  followers = fuzzificationFollowers()
  engageRate = fuzzificationEngageRate()
  concs = []

  for i in range(len(followers)):
    rules = [[followers[i][0],engageRate[i][0]],
             [followers[i][0],engageRate[i][1]],
             [followers[i][0],engageRate[i][2]],
             [followers[i][1],engageRate[i][0]],
             [followers[i][1],engageRate[i][1]],
             [followers[i][1],engageRate[i][2]],
             [followers[i][2],engageRate[i][0]],
             [followers[i][2],engageRate[i][1]],
             [followers[i][2],engageRate[i][2]]]

    for j in range(len(rules)):
      minn = min(rules[j])
      rules[j].append(minn)
    pilih = [[rules[0][2],rules[1][2],rules[3][2]],
             [rules[2][2],rules[4][2],rules[5][2],rules[6][2],rules[7][2]],
             [rules[8][2]]]
    
    medium = max(pilih[0])
    micro = max(pilih[1])
    nano = max(pilih[2])
    conc = []
    conc.append(medium)
    conc.append(micro)
    conc.append(nano)
    concs.append(conc)

    # print("INI rules nya")
    # for k in range(len(rules)):
    #   print(rules[k])
  print("HASIL Inferensi")
  for k in range(len(concs)):
      print(concs[k])

inferensi()


HASIL Inferensi
[0, 0, 1]
[0.12, 0.73, 0]
[0, 0, 1]
[0, 1, 0.05]
[0, 0.75, 0.4]


In [0]:
# for j in range(3):
    #   rules.append(followers[i][0])
    # for j in range(3):
    #   rules.append(followers[i][1])
    # for j in range(3):
    #   rules.append(followers[i][2])
      
    # rules[0].append(engageRate[i][0])
    # rules[1].append(engageRate[i][1])
    # rules[2].append(engageRate[i][3])

# **DEFUZZIFIKASI**

Using Takagi-Sugeno-style

In [0]:
def defuzzification():
  inf = inferensi()

  for i in range(len(inferensi)):
    


# **SORTING**

In [0]:
def sort():
  